Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [23]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
        
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    weights_2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    tf_beta = tf.Variable(0.1)
    
    def matopt(dataset, weights, biases):
        return tf.matmul(dataset, weights)+biases
    
    mat_temp = matopt(tf_train_dataset, weights_1, biases_1)
    relu_temp = tf.nn.relu(mat_temp)
    logits = matopt(relu_temp, weights_2, biases_2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += 0.01*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at steop %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % (accuracy(predictions, batch_labels)))
            print("Validation accuracy: %.1f%%" % (accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Initialized
Minibatch loss at steop 0: 330.425720
Minibatch accuracy: 13.3%
Validation accuracy: 28.2%
Minibatch loss at steop 500: 8.470108
Minibatch accuracy: 83.6%
Validation accuracy: 81.1%
Minibatch loss at steop 1000: 10.154570
Minibatch accuracy: 79.7%
Validation accuracy: 82.0%
Minibatch loss at steop 1500: 13.724258
Minibatch accuracy: 80.5%
Validation accuracy: 81.1%
Minibatch loss at steop 2000: 4.183042
Minibatch accuracy: 78.9%
Validation accuracy: 80.0%
Minibatch loss at steop 2500: 6.333561
Minibatch accuracy: 85.2%
Validation accuracy: 82.1%
Minibatch loss at steop 3000: 4.625069
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Test accuracy: 87.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
batch_size = 4
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
        
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    weights_2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    tf_beta = tf.Variable(0.1)
    
    def matopt(dataset, weights, biases):
        return tf.matmul(dataset, weights)+biases
    
    mat_temp = matopt(tf_train_dataset, weights_1, biases_1)
    relu_temp = tf.nn.relu(mat_temp)
    logits = matopt(relu_temp, weights_2, biases_2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += 0.01*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [10]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at steop %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % (accuracy(predictions, batch_labels)))
            print("Validation accuracy: %.1f%%" % (accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Initialized
Minibatch loss at steop 0: 226.547302
Minibatch accuracy: 25.0%
Validation accuracy: 13.5%
Minibatch loss at steop 500: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at steop 1000: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at steop 1500: nan
Minibatch accuracy: 50.0%
Validation accuracy: 10.0%
Minibatch loss at steop 2000: nan
Minibatch accuracy: 25.0%
Validation accuracy: 10.0%
Minibatch loss at steop 2500: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at steop 3000: nan
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Test accuracy: 10.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [24]:
batch_size = 128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
        
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size]))
    biases_1 = tf.Variable(tf.zeros([hidden_size]))
    weights_2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    tf_beta = tf.Variable(0.1)
    
    def matopt(dataset, weights, biases):
        return tf.matmul(dataset, weights)+biases
    
    mat_temp = matopt(tf_train_dataset, weights_1, biases_1)
    relu_temp = tf.nn.relu(mat_temp)
    tf.nn.dropout(relu_temp, .5)
    logits = matopt(relu_temp, weights_2, biases_2)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += 0.01*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [25]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at steop %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % (accuracy(predictions, batch_labels)))
            print("Validation accuracy: %.1f%%" % (accuracy(valid_prediction.eval(), valid_labels)))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Initialized
Minibatch loss at steop 0: 3514.844482
Minibatch accuracy: 13.3%
Validation accuracy: 32.2%
Minibatch loss at steop 500: 21.169329
Minibatch accuracy: 90.6%
Validation accuracy: 84.8%
Minibatch loss at steop 1000: 1.014608
Minibatch accuracy: 77.3%
Validation accuracy: 84.3%
Minibatch loss at steop 1500: 0.701028
Minibatch accuracy: 85.2%
Validation accuracy: 83.8%
Minibatch loss at steop 2000: 0.745450
Minibatch accuracy: 83.6%
Validation accuracy: 83.9%
Minibatch loss at steop 2500: 0.718124
Minibatch accuracy: 82.0%
Validation accuracy: 84.2%
Minibatch loss at steop 3000: 0.828817
Minibatch accuracy: 82.8%
Validation accuracy: 84.1%
Test accuracy: 90.4%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [134]:
batch_size = 64
hidden_size_1 = 1024
hidden_size_2 = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
        
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_size_1]))
    biases_1 = tf.Variable(tf.zeros([hidden_size_1]))
    weights_2 = tf.Variable(tf.truncated_normal([hidden_size_1, hidden_size_2]))
    biases_2 = tf.Variable(tf.zeros([hidden_size_2]))
    weights_3 = tf.Variable(tf.truncated_normal([hidden_size_2, num_labels]))
    biases_3 = tf.Variable(tf.zeros([num_labels]))
    
    
    def matopt(dataset, weights, biases):
        return tf.matmul(dataset, weights)+biases
    
    mat_temp_1 = matopt(tf_train_dataset, weights_1, biases_1)
    relu_temp_1 = tf.nn.relu(mat_temp_1)
    tf.nn.dropout(relu_temp_1, .5)
    mat_temp_2 = matopt(relu_temp_1, weights_2, biases_2)
    relu_temp_2 = tf.nn.relu(mat_temp_2)
    def sw():
        tf.nn.dropout(relu_temp_2, .5)
        logits = matopt(relu_temp_2, weights_3, biases_3)
    logits = matopt(relu_temp_2, weights_3, biases_3)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    loss += 0.01*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(weights_3))
    
    def opt(learn_var = 0.05):
        global optimizer
        optimizer = tf.train.GradientDescentOptimizer(learn_var).minimize(loss)
    opt()    
        
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        matopt(tf.nn.relu(matopt(tf.nn.relu(matopt(tf_valid_dataset, weights_1, biases_1)), weights_2, biases_2)), weights_3, biases_3))
        #tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2), weights_3) + biases_3)
    test_prediction = tf.nn.softmax(
        matopt(tf.nn.relu(matopt(tf.nn.relu(matopt(tf_test_dataset, weights_1, biases_1)), weights_2, biases_2)), weights_3, biases_3))
        #tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2), weights_3) + biases_3)

In [135]:
num_steps = 50001
switch=0
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    opt()
    print("Initialized")
    for step in range(num_steps):

        if step == 20000:
            sw()
        if step == 10000:
            opt(0.01)
        if step == 20000:
            opt(0.001)
            
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at steop %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % (accuracy(predictions, batch_labels)))
            print("Validation accuracy: %.1f%%" % (accuracy(valid_prediction.eval(), valid_labels)))
        if l<.4:
            break
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Initialized
Minibatch loss at steop 0: 6042.964355
Minibatch accuracy: 9.4%
Validation accuracy: 21.0%
Minibatch loss at steop 500: 2235.028320
Minibatch accuracy: 10.9%
Validation accuracy: 17.0%
Minibatch loss at steop 1000: 1356.053711
Minibatch accuracy: 29.7%
Validation accuracy: 21.8%
Minibatch loss at steop 1500: 823.198853
Minibatch accuracy: 28.1%
Validation accuracy: 31.1%
Minibatch loss at steop 2000: 499.788666
Minibatch accuracy: 50.0%
Validation accuracy: 47.1%
Minibatch loss at steop 2500: 303.231781
Minibatch accuracy: 65.6%
Validation accuracy: 62.4%
Minibatch loss at steop 3000: 183.845505
Minibatch accuracy: 79.7%
Validation accuracy: 76.2%
Minibatch loss at steop 3500: 111.848534
Minibatch accuracy: 78.1%
Validation accuracy: 79.6%
Minibatch loss at steop 4000: 67.933395
Minibatch accuracy: 84.4%
Validation accuracy: 82.1%
Minibatch loss at steop 4500: 41.341507
Minibatch accuracy: 89.1%
Validation accuracy: 83.4%
Minibatch loss at steop 5000: 25.393688
Minibatch ac